In [2]:
import os
import cv2
import pandas as pd
import easyocr


# Path to images
IMAGE_DIR = "C:/Users/nazih/OneDrive/Desktop/Decoding-Audio-Captchas/images"


In [3]:

# Initialize EasyOCR reader (for English language, you can add other languages if needed)
reader = easyocr.Reader(['en'])

# Path to the image you want to process
image_path ="C:/Users/nazih/Desktop/Decoding-Audio-Captchas/images/captcha_9984.png"
# Extract text from the image
result = reader.readtext(image_path, detail=0)  # detail=0 returns just the text

# Print extracted text
print("Extracted Text:", result)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Extracted Text: ['idGTid']


In [5]:


# Initialize EasyOCR reader (for English language)
reader = easyocr.Reader(['en'])

# Path to the folder containing your images
image_folder = 'C:/Users/nazih/Desktop/Decoding-Audio-Captchas/images'

# List all image files in the folder (you can filter by image extensions like .png, .jpg)
image_paths = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith(('.png'))]

# Extract text from each image and store in a list
extracted_texts = []

for image_path in image_paths:
    result = reader.readtext(image_path, detail=0)  # detail=0 returns just the text
    extracted_texts.append({'image_path': image_path, 'extracted_text': result[0] if result else ''})

# Create a DataFrame
df = pd.DataFrame(extracted_texts)

# Save DataFrame to CSV
csv_file_path = 'extracted_texts.csv'
df.to_csv(csv_file_path, index=False)

print(f"CSV file saved at: {csv_file_path}")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


CSV file saved at: extracted_texts.csv


In [13]:



import os
import pandas as pd


# Path to the image and audio files
image_folder = 'C:/Users/nazih/Desktop/Decoding-Audio-Captchas/images'
audio_folder = 'C:/Users/nazih/Desktop/Decoding-Audio-Captchas/audio'

# Read the CSV containing the extracted text from images
extracted_df = pd.read_csv('extracted_texts.csv')

# List all audio files in the audio folder (assuming they have the same name as image files)
audio_files = [f for f in os.listdir(audio_folder) if f.endswith(('.wav', '.mp3'))]

# Create a mapping for audio files to labels (assuming audio and image files share the same names)
audio_to_text_mapping = []

for image_file in extracted_df['image_path']:
    # Extract the file name (without extension) to match audio files
    base_name = os.path.basename(image_file).split('.')[0]  # Get the name without the extension
    audio_file = f"{base_name}.wav"  # Adjust this if audio file format is different (e.g., .mp3)

    # Check if the corresponding audio file exists
    if audio_file in audio_files:
        label = extracted_df[extracted_df['image_path'] == image_file]['extracted_text'].values[0]
        audio_to_text_mapping.append({'audio_filename': audio_file, 'extracted_text': label})

# Create a DataFrame from the mapping
mapping_df = pd.DataFrame(audio_to_text_mapping)

# Save the DataFrame to CSV
csv_file_path = 'audio_to_text_mapping.csv'
mapping_df.to_csv(csv_file_path, index=False)

print(f"CSV file with audio file names and labels saved at: {csv_file_path}")



CSV file with audio file names and labels saved at: audio_to_text_mapping.csv


In [11]:

# Load dataset
dataset_path = "audio_to_text_mapping.csv"
df = pd.read_csv(dataset_path)

# Print dataset preview
print(df.head())


     audio_filename extracted_text
0  captcha_0001.wav         ABOr4z
1  captcha_0002.wav         XLMbrT
2  captcha_0003.wav         bBRG5o
3  captcha_0004.wav         fdzhYy
4  captcha_0005.wav         SvOUfs


In [48]:
import whisper
import re

# Load Whisper model
model = whisper.load_model('base')

# Load and process the audio
audio = whisper.load_audio("C:/Users/nazih/Desktop/Decoding-Audio-Captchas/audio/captcha_6000.wav")
audio = whisper.pad_or_trim(audio)

# Create Mel spectrogram and detect language
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# Perform the transcription
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# Print the detected language
print(f"Detected Language: {result.language}")

# Access the best transcription (most likely transcription)
print("Transcription text:", result.text)

# If you want alternatives (if show_all=True was enabled)
if hasattr(result, 'segments'):
    for segment in result.segments:
        print(f"Alternative: {segment['text']} with confidence: {segment['confidence']}")

# Process the result text to handle 'small' and 'capital' modifiers
def process_text(text):
    words = text.split()
    processed_text = ""
    state = 'normal'  # Start with 'normal' state

    for word in words:
        if state == 'normal':
            if word.lower() == 'small':
                state = 'expecting_lower'
            elif word.lower() == 'capital':
                state = 'expecting_upper'
            else:
                processed_text += word
        elif state == 'expecting_lower':
            processed_text += word.lower()
            state = 'normal'
        elif state == 'expecting_upper':
            processed_text += word.upper()
            state = 'normal'
    processed_text = processed_text.replace(",", "").strip()
    processed_text = processed_text.replace(".", "").strip()
    return processed_text

# Process and correct the transcription
processed_text =process_text(result.text)
print("Processed text:", processed_text)


Detected Language: en
Transcription text: capital C, small G, small D, 8, small A, small R.
Processed text: Cgd8ar


In [15]:
import whisper
import os
import pandas as pd
import editdistance

# Function to process the text from Whisper's output
def process_text(text):
    words = text.split()
    processed_text = ""
    state = 'normal'  # Start with 'normal' state

    for word in words:
        if state == 'normal':
            if word.lower() == 'small':
                state = 'expecting_lower'
            elif word.lower() == 'capital':
                state = 'expecting_upper'
            else:
                processed_text += word
        elif state == 'expecting_lower':
            processed_text += word.lower()
            state = 'normal'
        elif state == 'expecting_upper':
            processed_text += word.upper()
            state = 'normal'

    return processed_text.replace(",", "").replace(".", "").strip()

# Function to load the correct texts from CSV
def load_correct_texts(csv_file):
    # Load the CSV into a pandas dataframe
    df = pd.read_csv(csv_file)
    return {row['audio_filename']: row['extracted_text'] for _, row in df.iterrows()}

# Function to calculate character error rate (CER)
def calculate_cer(reference, hypothesis):
    reference = reference.replace(" ", "")  # Remove spaces for CER calculation
    hypothesis = hypothesis.replace(" ", "")  # Remove spaces for CER calculation
    return editdistance.eval(reference, hypothesis) / len(reference)

# Function to calculate accuracy
def calculate_accuracy(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()
    correct = sum([1 for r, h in zip(reference_words, hypothesis_words) if r == h])
    return correct / len(reference_words)

# Transcription and comparison with the correct texts
def transcribe_and_evaluate(audio_directory, csv_file):
    # Load the Whisper model
    model = whisper.load_model("base")
    
    # Load the correct texts from CSV
    correct_texts = load_correct_texts(csv_file)

    # List all files in the audio directory
    files = [f for f in os.listdir(audio_directory) if f.endswith('.wav')]
    
    transcribed_texts = []
    correct_texts_list = []
    
    # Process each file
    for filename in files:
        file_path = os.path.join(audio_directory, filename)
        
        # Preprocess the audio
        audio = whisper.load_audio(file_path)
        audio = whisper.pad_or_trim(audio)
        
        # Perform the transcription
        result = model.transcribe(audio, fp16=False)
        processed_text = process_text(result['text'])
        
        # Get the correct transcription from CSV
        correct_text = correct_texts.get(filename, None)
        
        if correct_text:
            # Add to the list of transcribed and correct texts for CER and accuracy calculation
            transcribed_texts.append(processed_text)
            correct_texts_list.append(correct_text)
            
            # Calculate and print CER for each audio file
            cer = calculate_cer(correct_text, processed_text)
            accuracy = calculate_accuracy(correct_text, processed_text)
            print(correct_text, processed_text)
            print(f"CER for {filename}: {cer * 100:.2f}%")
            print(f"Accuracy for {filename}: {accuracy * 100:.2f}%")
        else:
            print(f"Warning: Correct text for {filename} not found in CSV.")
    
    return transcribed_texts

# Path to the directory containing your audio files
audio_directory = "C:/Users/nazih/Desktop/Decoding-Audio-Captchas/audio"

# Path to your CSV file containing correct texts
csv_file = "C:/Users/nazih/Desktop/Decoding-Audio-Captchas/audio_to_text_mapping.csv"

# Run the transcription and evaluation
transcriptions = transcribe_and_evaluate(audio_directory, csv_file)


ABOr4z BBOr4z
CER for captcha_0001.wav: 16.67%
Accuracy for captcha_0001.wav: 0.00%
xLMbrT xLMbrT
CER for captcha_0002.wav: 0.00%
Accuracy for captcha_0002.wav: 100.00%
bBRG5o bBRGFineZero
CER for captcha_0003.wav: 116.67%
Accuracy for captcha_0003.wav: 0.00%
fd2hYy fb2hYy
CER for captcha_0004.wav: 16.67%
Accuracy for captcha_0004.wav: 0.00%
SvOUfs andv0Ufs
CER for captcha_0005.wav: 66.67%
Accuracy for captcha_0005.wav: 0.00%
86dHWG 8dEDGEWG
CER for captcha_0006.wav: 83.33%
Accuracy for captcha_0006.wav: 0.00%
hHrZVn edgeHr7Vn
CER for captcha_0007.wav: 83.33%
Accuracy for captcha_0007.wav: 0.00%
hmRfZv hmRfZv
CER for captcha_0008.wav: 0.00%
Accuracy for captcha_0008.wav: 100.00%
wYDOet wYDZimroat
CER for captcha_0009.wav: 100.00%
Accuracy for captcha_0009.wav: 0.00%
7gIAHO gIAEDGEL
CER for captcha_0010.wav: 100.00%
Accuracy for captcha_0010.wav: 0.00%
vutDgy vutDgY
CER for captcha_0011.wav: 16.67%
Accuracy for captcha_0011.wav: 0.00%
jJdijT jJdijT
CER for captcha_0012.wav: 0.00%
Accura

KeyboardInterrupt: 